**RQ2: Based on past trends, what are the projected self-rated poverty levels for the next decade?**

**Importing Necessary Libraries**

In [1]:

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

**Plot and Design Setting**

In [2]:
# Color settings
colors = ["#648FFF", "#785EF0", "#A11C5D", "#FE6100", "#FFB000", "#000000", "#FFFFFF"]
colors_grad = sns.color_palette('flare_r', 12)
colors_heat1 = sns.color_palette('flare_r', as_cmap=True)
colors_heat2 = sns.diverging_palette(315, 261, s=74, l=50, center='dark', as_cmap=True)
color_bg = "#1B181C"
color_text = "#FFFFFF"

# Plot settings
mpl.rcParams['figure.dpi'] = 600
mpl.rcParams["figure.figsize"] = (19.2, 10.8)
mpl.rcParams['font.family'] = 'Roboto'
mpl.rcParams['figure.titlesize'] = 32
mpl.rcParams['axes.titlesize'] = 32
mpl.rcParams['axes.titleweight'] = 'bold'
mpl.rcParams['axes.labelsize'] = 22
mpl.rcParams['xtick.labelsize'] = 22
mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['axes.titlepad'] = 72
mpl.rcParams['axes.labelpad'] = 10
mpl.rcParams['xtick.major.pad'] = 10
mpl.rcParams['ytick.major.pad'] = 10
mpl.rcParams['xtick.major.width'] = 0
mpl.rcParams['xtick.minor.width'] = 0
mpl.rcParams['ytick.major.width'] = 0
mpl.rcParams['ytick.minor.width'] = 0
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.grid'] = False
mpl.rcParams['legend.title_fontsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['legend.frameon'] = False
plt.rcParams['patch.linewidth'] = 0
plt.rcParams['patch.edgecolor'] = 'none'
mpl.rcParams["figure.facecolor"] = color_bg
mpl.rcParams["axes.facecolor"] = color_bg
mpl.rcParams["savefig.facecolor"] = color_bg
mpl.rcParams['text.color'] = color_text
mpl.rcParams['axes.labelcolor'] = color_text
mpl.rcParams['xtick.color'] = color_text
mpl.rcParams['ytick.color'] = color_text
mpl.rcParams['axes.edgecolor'] = color_text

**Data Preprocessing and Extraction**

In [3]:
# Load Dataset and Data Cleaning
poverty_data = pd.read_csv("CS132_DataSheet-graph_data.csv")
poverty_data['Year'] = poverty_data['Year'].ffill() + (poverty_data['Month Only'] / 12)

In [4]:
# Extract data
x = poverty_data['Year']
y_poor = poverty_data['Poor Respondent Count']
y_not_poor = poverty_data['Not Poor Respondent Count']
y_borderline = poverty_data['Borderline Respondent Count']

**Data Standardization**

In [5]:
# Data Split and Reshape
# Poor
X_train_poor, X_test_poor, Y_train_poor, Y_test_poor = train_test_split(x, y_poor, test_size=0.5, random_state=42)
X_train_poor = np.array(X_train_poor).reshape(-1, 1)
X_test_poor = np.array(X_test_poor).reshape(-1, 1)
Y_test_poor = np.array(Y_test_poor)

# Not Poor
X_train_not_poor, X_test_not_poor, Y_train_not_poor, Y_test_not_poor = train_test_split(x, y_not_poor, test_size=0.5, random_state=42)
X_train_not_poor = np.array(X_train_not_poor).reshape(-1, 1)
X_test_not_poor = np.array(X_test_not_poor).reshape(-1, 1)
Y_test_not_poor = np.array(Y_test_not_poor)

# Borderline
X_train_borderline, X_test_borderline, Y_train_borderline, Y_test_borderline = train_test_split(x, y_borderline, test_size=0.5, random_state=42)
X_train_borderline = np.array(X_train_borderline).reshape(-1, 1)
X_test_borderline = np.array(X_test_borderline).reshape(-1, 1)
Y_test_borderline = np.array(Y_test_borderline)

**Regrssion Model Training**

In [6]:
# Regression Models
# Poor
model_poor = linear_model.LinearRegression()
model_poor.fit(X_train_poor, Y_train_poor)
Y_pred_poor = model_poor.predict(X_test_poor)

# Not Poor
model_not_poor = linear_model.LinearRegression()
model_not_poor.fit(X_train_not_poor, Y_train_not_poor)
Y_pred_not_poor = model_not_poor.predict(X_test_not_poor)

# Borderline
model_borderline = linear_model.LinearRegression()
model_borderline.fit(X_train_borderline, Y_train_borderline)
Y_pred_borderline = model_borderline.predict(X_test_borderline)

# Polynomial Regression Models
degree = 3  # Degree of the polynomial

# Poor
poly_model_poor = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
poly_model_poor.fit(X_train_poor, Y_train_poor)
y_pred_poor_poly = poly_model_poor.predict(X_test_poor)

# Not Poor
poly_model_not_poor = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
poly_model_not_poor.fit(X_train_not_poor, Y_train_not_poor)
y_pred_not_poor_poly = poly_model_not_poor.predict(X_test_not_poor)

# Borderline
poly_model_borderline = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
poly_model_borderline.fit(X_train_borderline, Y_train_borderline)
y_pred_borderline_poly = poly_model_borderline.predict(X_test_borderline)

**Regression Metrics**

In [7]:
# Print Regression Metrics
print("Poor:")
print(f"Coefficient: {model_poor.coef_[0]:.2f}")
print(f"Intercept: {model_poor.intercept_:.2f}")
print(f"Mean Squared Error: {mean_squared_error(Y_test_poor, Y_pred_poor):.2f}")
print(f"R-squared: {r2_score(Y_test_poor, Y_pred_poor):.2f}")
print(f"Mean Squared Error (Poly): {mean_squared_error(Y_test_poor, y_pred_poor_poly):.2f}")
print(f"R-squared (Poly): {r2_score(Y_test_poor, y_pred_poor_poly):.2f}")
print("\nNot Poor:")
print(f"Coefficient: {model_not_poor.coef_[0]:.2f}")
print(f"Intercept: {model_not_poor.intercept_:.2f}")
print(f"Mean Squared Error: {mean_squared_error(Y_test_not_poor, Y_pred_not_poor):.2f}")
print(f"R-squared: {r2_score(Y_test_not_poor, Y_pred_not_poor):.2f}")
print(f"Mean Squared Error (Poly): {mean_squared_error(Y_test_not_poor, y_pred_not_poor_poly):.2f}")
print(f"R-squared (Poly): {r2_score(Y_test_not_poor, y_pred_not_poor_poly):.2f}")
print("\nBorderline:")
print(f"Coefficient: {model_borderline.coef_[0]:.2f}")
print(f"Intercept: {model_borderline.intercept_:.2f}")
print(f"Mean Squared Error: {mean_squared_error(Y_test_borderline, Y_pred_borderline):.2f}")
print(f"R-squared: {r2_score(Y_test_borderline, Y_pred_borderline):.2f}")
print(f"Mean Squared Error (Poly): {mean_squared_error(Y_test_borderline, y_pred_borderline_poly):.2f}")
print(f"R-squared (Poly): {r2_score(Y_test_borderline, y_pred_borderline_poly):.2f}")

Poor:
Coefficient: -6.12
Intercept: 12928.45
Mean Squared Error: 4640.55
R-squared: 0.50
Mean Squared Error (Poly): 3776.55
R-squared (Poly): 0.59

Not Poor:
Coefficient: 4.05
Intercept: -7892.84
Mean Squared Error: 2608.98
R-squared: 0.52
Mean Squared Error (Poly): 2623.14
R-squared (Poly): 0.52

Borderline:
Coefficient: 2.06
Intercept: -3835.61
Mean Squared Error: 6611.97
R-squared: 0.02
Mean Squared Error (Poly): 5927.41
R-squared (Poly): 0.12


**Data Plotting**

In [11]:
# Create the combined plot
plt.figure()
plt.figure(figsize=(19.2, 10.8)) 
# Scatter plots with smaller and more transparent circles
scatter_poor = plt.scatter(x, y_poor, color=colors[0], alpha=0.5)  # Smaller size and lower alpha
scatter_not_poor = plt.scatter(x, y_not_poor, color=colors[2], alpha=0.5)
scatter_borderline = plt.scatter(x, y_borderline, color=colors[4], alpha=0.5)

# Predict values for the extended range using polynomial models
x_extended_range = np.linspace(x.min(), x.max() + 10, 100).reshape(-1, 1)
y_pred_poor_extended_poly = poly_model_poor.predict(x_extended_range)
y_pred_not_poor_extended_poly = poly_model_not_poor.predict(x_extended_range)
y_pred_borderline_extended_poly = poly_model_borderline.predict(x_extended_range)

# Plot polynomial regression lines (Hide the label)
plt.plot(x_extended_range, y_pred_poor_extended_poly, color=colors[0], linestyle='--')
plt.plot(x_extended_range, y_pred_not_poor_extended_poly, color=colors[2], linestyle='--')
plt.plot(x_extended_range, y_pred_borderline_extended_poly, color=colors[4], linestyle='--')

# Events with specified color pattern
# 1986 People Power Revolution
specific_year = 1986
plt.axvline(x=specific_year, color=colors_grad[2], linestyle='--')
plt.annotate(
    f'1986 People Power Revolution',
    xy=(specific_year, 900),
    xytext=(specific_year-1, 900),
    arrowprops=dict(facecolor=colors_grad[2], arrowstyle='->', lw=0.8),
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[2])
)

# 1991 Mount Pinatubo Eruption
specific_year = 1991
plt.axvline(x=specific_year, color=colors_grad[4], linestyle='--')
plt.annotate(
    f'1991 Mount Pinatubo Eruption',
    xy=(specific_year, 800),
    xytext=(specific_year-1, 800),
    arrowprops=dict(facecolor=colors_grad[4], arrowstyle='->', lw=0.8),
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[4])
)

# 1997 Asian Financial Crisis
specific_year = 1997
plt.axvline(x=specific_year, color=colors_grad[5], linestyle='--')
plt.annotate(
    f'1997 Asian Financial Crisis',
    xy=(specific_year, 700),
    xytext=(specific_year-1, 700),
    arrowprops=dict(facecolor=colors_grad[5], arrowstyle='->', lw=0.8),
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[5])
)

# 2008 Global Financial Crisis
specific_year = 2008
plt.axvline(x=specific_year, color=colors_grad[6], linestyle='--')
plt.annotate(
    f'2008 Global Financial Crisis',
    xy=(specific_year, 600),
    xytext=(specific_year-1, 600),
    arrowprops=dict(facecolor=colors_grad[6], arrowstyle='->', lw=0.8),
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[6])
)

# 2020 COVID 19 Pandemic
specific_year = 2020
plt.axvline(x=specific_year, color=colors_grad[7], linestyle='--')  # Keep the line color as is
plt.annotate(
    f'2020 COVID 19 Pandemic',
    xy=(specific_year, 500),
    xytext=(specific_year-1, 500),
    arrowprops=dict(facecolor=colors_grad[7], arrowstyle='->', lw=0.8),
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[7]),
)

# 2023 Post Pandemic Recovery
specific_year = 2023
plt.axvline(x=specific_year, color=colors_grad[8], linestyle='--')
plt.annotate(
    f'2023 Post Pandemic Recovery',
    xy=(specific_year, 400),
    xytext=(specific_year-1, 400),
    arrowprops=dict(facecolor=colors_grad[8], arrowstyle='->', lw=0.8),
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[8])
)

# Predicting future values
specific_year = 2026
plt.axvline(x=specific_year, color=colors_grad[10], linestyle='--', )
plt.annotate(
    f'Prediction for self-rated poverty',
    xy=(specific_year, 900),
    xytext=(specific_year-1, 900),  # Adjust position for clarity
    arrowprops=dict(facecolor=colors_grad[10], arrowstyle='->', lw=0.8),  # Thinner arrow
    fontsize=10,  # Smaller font size
    bbox=dict(boxstyle='round,pad=0.2', edgecolor='white', facecolor=colors_grad[10]),  # Smaller text box
    #Black color for the text
    color='black'  # Change text color to black 
)


# Legend for scatter points
legend1 = plt.legend(handles=[scatter_poor, scatter_not_poor, scatter_borderline],
                     labels=['Poor', 'Not Poor', 'Borderline'],
                     title='Data',
                     loc='upper left',
                     bbox_to_anchor=(1.05, 1.0))
plt.gca().add_artist(legend1)

# Define dummy lines for the legend since linear regression lines are commented out
line_poor = plt.Line2D([0], [0], color=colors[0], linestyle='--', label='Poor (Poly)')
line_not_poor = plt.Line2D([0], [0], color=colors[2], linestyle='--', label='Not Poor (Poly)')
line_borderline = plt.Line2D([0], [0], color=colors[4], linestyle='--', label='Borderline (Poly)')

# Legend for trend lines
legend2 = plt.legend(handles=[line_poor, line_not_poor, line_borderline],
                     labels=['Poor', 'Not Poor', 'Borderline'],
                     title='Trends',
                     loc='upper left',
                     bbox_to_anchor=(1.05, 0.7))
plt.gca().add_artist(legend2)

# Customize the plot
plt.xlabel('Year')
plt.ylabel('Respondent Count')
plt.title('Poverty Respondent Counts Over Time', fontdict={'family': 'Roboto', 'weight': 'bold'})
plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1.05))

# Adjust the figure margins to accommodate the legends
plt.subplots_adjust(right=1)

# Adjust layout to prevent clipping
plt.tight_layout()
plt.savefig('nutshell_plot.pdf')
plt.show()
# Save the plot


/var/folders/ql/g_97vpms3rv3x6p09f6dt0h40000gn/T/ipykernel_20557/1809591418.py:133: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1.05))


<Figure size 11520x6480 with 0 Axes>